In [1]:
import numpy as np
import polars as pl
from ebnerd_benchmark_lib.utils._constants import DEFAULT_USER_COL

import os
from pathlib import Path


data_base = Path(os.getcwd()).parent / "data-merged" / "merged"
train_val_base = data_base / "1-ebnerd_demo_(20MB)"
# train_val_base = Path(os.getcwd()).parent / "data-merged" / "merged" / "3-ebnerd_large_(3.0GB)"
test_base = data_base / "5-ebnerd_testset_(1.5GB)"
assert train_val_base.exists() and test_base.exists()


"""
read in user data
"""

tb = pl.scan_parquet(train_val_base / "train" / "behaviors.parquet")
th = pl.scan_parquet(train_val_base / "train" / "history.parquet")
assert list(set(tb.columns).intersection(set(th.columns)))[0] == DEFAULT_USER_COL
train_user: pl.LazyFrame = tb.join(th, on=DEFAULT_USER_COL, how="inner")

vb: pl.LazyFrame = pl.scan_parquet(train_val_base / "validation" / "behaviors.parquet")
vh: pl.LazyFrame = pl.scan_parquet(train_val_base / "validation" / "history.parquet")
assert list(set(vb.columns).intersection(set(vh.columns)))[0] == DEFAULT_USER_COL
val_user: pl.LazyFrame = vb.join(vh, on=DEFAULT_USER_COL, how="inner")

trb: pl.LazyFrame = pl.scan_parquet(test_base / "test" / "behaviors.parquet")
trh: pl.LazyFrame = pl.scan_parquet(test_base / "test" / "history.parquet")
assert list(set(trb.columns).intersection(set(trh.columns)))[0] == DEFAULT_USER_COL
test_user: pl.LazyFrame = trb.join(trh, on=DEFAULT_USER_COL, how="inner")


"""
read in article data
"""

train_articles: pl.LazyFrame = pl.scan_parquet(train_val_base / "articles.parquet")
val_articles: pl.LazyFrame = train_articles
test_articles: pl.LazyFrame = pl.scan_parquet(test_base / "articles.parquet")

articles_word2vec: pl.LazyFrame = pl.scan_parquet(data_base / "7-Ekstra-Bladet-word2vec_(133MB)" / "document_vector.parquet")
articles_image_embeddings: pl.LazyFrame = pl.scan_parquet(data_base / "8-Ekstra_Bladet_image_embeddings_(372MB)" / "image_embeddings.parquet")
articles_contrastive_vector: pl.LazyFrame = pl.scan_parquet(data_base / "9-Ekstra-Bladet-contrastive_vector_(341MB)" / "contrastive_vector.parquet")
articles_bert_base_multilingual_cased: pl.LazyFrame = pl.scan_parquet(data_base / "10-google-bert-base-multilingual-cased_(344MB)" / "bert_base_multilingual_cased.parquet")
articles_xlm_roberta_base: pl.LazyFrame = pl.scan_parquet(data_base / "11-FacebookAI-xlm-roberta-base_(341MB)" / "xlm_roberta_base.parquet")

In [2]:
train_user.head(5).collect()

# num rows
train_user.len().collect()

AttributeError: 'LazyFrame' object has no attribute 'len'

In [23]:
# def print_missing_vals(df: pl.LazyFrame):
#     # print num cols
